In [1]:
########## import libraries ############
import numpy as np
import pandas as pd
from numpy import array
from numpy import asarray
from numpy import zeros
import matplotlib as mpl
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Embedding
import IPython; from IPython.display import display, HTML
def dfPrint(df):
    display(HTML(df.to_html()))
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cardioid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using TensorFlow backend.


In [2]:
############# import data ##############
def import_data():
    train_path = "./data/nlp_trip_advisor/train.csv"
    data = pd.read_csv(train_path)
    data = data[data.Is_Response.isnull() == False]
#     data['Is_Response'] = data['Is_Response'].map(int)
    data = data[data['Description'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    print ('dataset loaded with shape', data.shape)
    data = pd.get_dummies(data, columns=["Is_Response"])
    data.drop(['Browser_Used', 'Device_Used', 'Is_Response_not happy'], axis=1, inplace=True)
    data.columns = ['User_ID', "Description", "Sentiment"]
    return data

data = import_data()
dfPrint(data.sample(5))

dataset loaded with shape (38932, 5)


,User_ID,Description,Sentiment
11265,id21591,I stayed at this hotel on a Friday and Saturda...,1
9079,id19405,Beautiful hotel directly on the harbor. Checke...,1
16965,id27291,Enjoyable stay in SF. Hyatt was lovely and the...,1
24743,id35069,This was my first time staying in a Staybridge...,0
11374,id21700,I just stayed - nights at the Parc --. Great l...,1


In [3]:
################ clean data ##############
def clean_decription(line):
    words = list(set(line.split()))
    words = list(sorted([x.lower() for x in words]))
#     print (len(words))
#     print (words)
    words2 = words.copy()
    for word in list(set(words2).intersection(stop_words)):
        words.remove(word)
#     out = ' '.join(words)
    out = ' '.join(e for e in words if e.isalnum())
    if out == "":
        print(words)
#     table = str.maketrans({key: None for key in string.punctuation})
#     out.translate(table, string.punctuation)
    return out
data["Description"] = data["Description"].map(clean_decription)
#     print (len(words))
#     print (words)
#     break
# data

In [4]:
############### define documents ################
docs = data["Description"]
############# define class labels ################
labels = data["Sentiment"]
############## prepare tokenizer #################
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
########## integer encode the documents ##########
encoded_docs = t.texts_to_sequences(docs)
########## pad documents to a max length ##########
max_length = 200
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')


In [5]:
########### load the whole embedding into memory ############
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for num, line in enumerate(f):
    values = line.split()
    word = values[0]
    if word in stop_words :
        print (word)
        continue
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

the
of
to
and
in
a
for
that
on
is
was
with
he
as
it
by
at
from
his
an
be
has
are
have
but
were
not
this
who
they
had
i
which
will
their
or
its
after
been
we
more
about
up
when
there
all
out
she
other
her
than
over
into
some
you
if
no
can
do
only
most
against
so
them
what
him
during
before
while
where
because
now
between
did
just
under
such
then
any
through
being
down
off
both
those
these
our
here
should
very
my
how
until
same
won
each
does
own
me
few
too
again
your
once
further
having
himself
why
am
doing
themselves
itself
above
whom
below
s
re
d
m
nor
t
herself
myself
don
y
ma
o
yourself
ourselves
haven
ours
theirs
shan
yours
hers
ain
ll
yourselves
ve
doesn
didn
isn
aren
wasn
couldn
hasn
shouldn
wouldn
weren
Loaded 399851 word vectors.


In [6]:
########### create a weight matrix for words in training docs ############
embedding_matrix = zeros((vocab_size, 100))
mil=0
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        mil+=1
        embedding_matrix[i] = embedding_vector
print(mil)

29657


In [7]:
################ define model#################
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=True)
model.add(e)
model.add(LSTM(25))
# model.add(Flatten())
model.add(Dense(50,  kernel_initializer="normal",activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(25,  kernel_initializer="normal",activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(15,  kernel_initializer="normal",activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

############## compile the model ##############
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc','binary_accuracy'])
############# summarize the model ##############
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 100)          3958600   
_________________________________________________________________
lstm_1 (LSTM)                (None, 25)                12600     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                1300      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dropout_2 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 15)                390       
__________

In [8]:
############### split into train test data #################
padded_docs_train, padded_docs_test, labels_train, labels_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=69)
################### fit the model ##########3#############
model.fit(padded_docs_train, labels_train, epochs=3, validation_split = 0.2,verbose=1)

Train on 24916 samples, validate on 6229 samples
Epoch 1/3
24916/24916 [==============================] - 358s 14ms/step - loss: 0.6345 - acc: 0.6789 - binary_accuracy: 0.6789 - val_loss: 0.6268 - val_acc: 0.6808 - val_binary_accuracy: 0.6808
Epoch 2/3
24916/24916 [==============================] - 366s 15ms/step - loss: 0.6298 - acc: 0.6793 - binary_accuracy: 0.6793 - val_loss: 0.6260 - val_acc: 0.6808 - val_binary_accuracy: 0.6808
Epoch 3/3
24916/24916 [==============================] - 368s 15ms/step - loss: 0.6288 - acc: 0.6793 - binary_accuracy: 0.6793 - val_loss: 0.6254 - val_acc: 0.6808 - val_binary_accuracy: 0.6808


In [9]:
# evaluate the model
loss, accuracy,binaryAccuracy = model.evaluate(padded_docs_test, labels_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))
print('binaryAccuracy: %f' % (binaryAccuracy*100))
predicted = model.predict(padded_docs_test)

7787/7787 [==============================] - 24s 3ms/step
Accuracy: 68.755618
binaryAccuracy: 68.755618
